In [1]:
from csp import *

Our function to formulate the problem. The variables are each clue (they are "variables" because they are relevant to the problem and participate in binary constraints, but their domain is only 1 value which is already given to us initially) as well as each row and column (their domain most of the time is all lists of permutations between 1 to the gridsize). The constraints graph (in the form of a dict) is constructed with each clue being connected to itself (trivial reflexive unary constraint which must be present to avoid errors in the AIMA machinery), with each row related to each column as well as its corresponding left and right clues, and finally each column connected to each row and its corresponding up and down clues.

In [19]:
def formulate(gridsize, leftclues, rightclues, upclues, downclues):
    N = gridsize
    assert len(leftclues) +\
           len(rightclues) +\
           len(upclues) +\
           len(downclues) == 4*N
    
    variables = []
    for i in range(1, N + 1):
        variables.append("L" + str(i))
        variables.append("R" + str(i))
        variables.append("U" + str(i))
        variables.append("D" + str(i))
        variables.append("S" + str(i))
        variables.append("C" + str(i))
    
    domains = {}
    left = right = up = down = 0
    for i in variables:
        if i[0] == "L":
            domains[i] = [leftclues[left]]
            left += 1
        elif i[0] == "R":
            domains[i] = [rightclues[right]]
            right += 1
        elif i[0] == "U":
            domains[i] = [upclues[up]]
            up += 1
        elif i[0] == "D":
            domains[i] = [downclues[down]]
            down += 1
        elif i[0] == "S":
            index = int(i[1]) - 1
            domains[i] = permutations(N, leftclues[index], rightclues[index])
        elif i[0] == "C":
            index = int(i[1]) - 1
            domains[i] = permutations(N, upclues[index], downclues[index])
        
    neighbors = {}
    row = col = 0
    for i in variables:
        if i[0] == "S":
            neighbors[i] = ["C" + str(j) for j in range(1, N + 1)]
            neighbors[i].append("L" + str(row + 1))
            neighbors[i].append("R" + str(row + 1))
            row += 1
        elif i[0] == "C":
            neighbors[i] = ["S" + str(j) for j in range(1, N + 1)]
            neighbors[i].append("U" + str(col + 1))
            neighbors[i].append("D" + str(col + 1))
            col += 1
        else:
            neighbors[i] = [i]
    
    return variables, domains, neighbors

def permutations(N, clue1, clue2):
    '''Assuming equal probability distribution of the clues (and no 0s given),
    there is a 2/n chance per row/column that a 1 comes up as a clue. When that
    happens we can reduce the domain of such row/column from n! permutations
    to (n - 1)! permutations. More reductions to the domain are possible by
    observing mathematical properties of the problem and with clever programming
    for generating the permutations; we only implemented the simpliest case/rule
    for domain reduction which is that when a clue is one, the highest skyscrapper
    has to be placed at that bystander point of view no matter what.'''
    if clue1 == 1:
        perms = list(itertools.permutations([i for i in range(1, N)]))
        perms = [list(i) for i in perms]
        for i in perms:
            i.insert(0, N)
        return [tuple(i) for i in perms]
    
    elif clue2 == 1:
        perms = list(itertools.permutations([i for i in range(1, N)]))
        perms = [list(i) for i in perms]
        for i in perms:
            i.append(N)
        return [tuple(i) for i in perms]
    
    else:
        return list(itertools.permutations([i for i in range(1, N + 1)]))
        


Our function for checking constraint satisfaction between a variable A and B with values of a & b repectively

In [14]:
def constraints(A, a, B, b):
    if (A[0] == "S" and B[0] == "C") or (A[0] == "C" and B[0] == "S"):
        #This case basically works like a crossword
        A_index = int(B[1]) - 1
        B_index = int(A[1]) - 1
        if a[A_index] == b[B_index]:
            return True
    
    elif (A[0] == "S" and B[0] == "L") or (A[0] == "C" and B[0] == "U"):
        #Check clue satisfaction from the perspective of a up/left bystander
        if b == 0:
            return True
        else:
            count = biggest = 0
            for i in a:
                if i > biggest:
                    biggest = i
                    count += 1
            if count == b:
                return True
                
    elif (A[0] == "S" and B[0] == "R") or (A[0] == "C" and B[0] == "D"):
        #Check clue satisfaction from the perspective of a right/down bystander
        if b == 0:
            return True
        else:
            a = list(a)
            a.reverse()
            count = biggest = 0
            for i in a:
                if i > biggest:
                    biggest = i
                    count += 1
            if count == b:
                return True
    
    else: #Unary reflexive constraint of the clues, trivially true by A == B
        return True

This is just an utility function to print a puzzle after it has been solved:

In [15]:
def printPuzzle(skyscrapper):
    if skyscrapper is None:
        print("Puzzle couldn't be solved")
        return
    
    clues = [ [] for i in range(1, 5)]
    for i in skyscrapper.keys():
        if i[0] == "L":
            clues[0].append(skyscrapper[i])
        elif i[0] == "R":
            clues[1].append(skyscrapper[i])
        elif i[0] == "U":
            clues[2].append(skyscrapper[i])
        elif i[0] == "D":
            clues[3].append(skyscrapper[i])
    
    mold = {}
    count = -1
    for i in skyscrapper.keys():
        if i[0] == 'S':
            count += 1
            mold[i] = list(skyscrapper[i])
            mold[i].insert(0, clues[0][count])
            mold[i].append(clues[1][count])
    
    for i in range(1, len(mold) + 1):
        if clues[0][i - 1] == 0 and clues[1][i - 1] == 0:
            for j in range(1, len(mold) + 1):
                if clues[2][j - 1] == 0 and clues[3][j - 1] == 0:
                    mold['S' + str(i)][j] = '-'
    
    N = len(clues[0])
    count = 0
    for i in clues[2]:
        count += 1
        if count == 1:
            print("   " + str(i) + "  ", end = "")
        elif count == N:
            print(" " + str(i))
        else:
            print(" " + str(i) + "  ", end = "")
            
    N = len(mold['S1'])
    for i in mold.keys():
        count = 0
        for j in mold[i]:
            count += 1
            if count == 1:
                print(str(j), end = " ")
            elif count == N:
                print(j)
                count = 0
            else:
                print("|" + str(j) + "| ", end = "")
                         
    N = len(clues[0])
    count = 0
    for i in clues[3]:
        count += 1
        if count == 1:
            print("   " + str(i) + "  ", end = "")
        elif count == N:
            print(" " + str(i))
        else:
            print(" " + str(i) + "  ", end = "")

In [18]:
#Formulate and solve assignment example
variables, domains, neighbors = formulate(5, [3, 2, 3, 2, 1], [3, 4, 1, 2, 2], [4, 2, 1, 2, 3], [1, 4, 3, 2, 2])
example_skyscrapper = CSP(variables, domains, neighbors, constraints)
solution_example = backtracking_search(example_skyscrapper)
printPuzzle(solution_example)

   4   2   1   2   3
3 |2| |3| |5| |4| |1| 3
2 |1| |5| |4| |3| |2| 4
3 |3| |4| |2| |1| |5| 1
2 |4| |2| |1| |5| |3| 2
1 |5| |1| |3| |2| |4| 2
   1   4   3   2   2


In [21]:
#Formulate and solve assignment challenge 1
variables, domains, neighbors = formulate(4, [1, 3, 2, 2], [3, 2, 1, 2], [1, 3, 2, 2], [3, 1, 2, 2])
skyscrapper_1 = CSP(variables, domains, neighbors, constraints)
solution_1 = backtracking_search(skyscrapper_1)
printPuzzle(solution_1)

   1   3   2   2
1 |4| |1| |3| |2| 3
3 |2| |3| |4| |1| 2
2 |3| |2| |1| |4| 1
2 |1| |4| |2| |3| 2
   3   1   2   2


In [22]:
#Formulate and solve assignment challenge 2
variables, domains, neighbors = formulate(4, [3, 1, 2, 3], [2, 2, 3, 1], [2, 2, 1, 3], [3, 2, 2, 1])
skyscrapper_2 = CSP(variables, domains, neighbors, constraints)
solution_2 = backtracking_search(skyscrapper_2)
printPuzzle(solution_2)

   2   2   1   3
3 |1| |3| |4| |2| 2
1 |4| |2| |1| |3| 2
2 |3| |4| |2| |1| 3
3 |2| |1| |3| |4| 1
   3   2   2   1


In [24]:
#Formulate and solve assignment challenge 3
variables, domains, neighbors = formulate(4, [3, 3, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 2, 0])
skyscrapper_3 = CSP(variables, domains, neighbors, constraints)
solution_3 = backtracking_search(skyscrapper_3)
printPuzzle(solution_3)

   0   0   1   0
3 |1| |2| |4| |3| 0
3 |2| |3| |1| |4| 0
0 |-| |-| |2| |-| 0
0 |-| |-| |3| |-| 0
   0   0   2   0


In [28]:
#Extra 6x6 challenge taken from http://www.brainbashers.com/showskyscraper.asp?date=1009&size=6&diff=1
#Using heuristics and forward checking helps here
variables, domains, neighbors = formulate(6, [2, 3, 1, 2, 3, 3],
                                             [3, 2, 2, 3, 3, 1],
                                             [3, 1, 3, 2, 2, 3],
                                             [3, 2, 2, 2, 3, 1])
skyscrapper_bonus = CSP(variables, domains, neighbors, constraints)
solution_bonus = backtracking_search(skyscrapper_bonus,
                        select_unassigned_variable=mrv,
                        order_domain_values=lcv,
                        inference=forward_checking)
printPuzzle(solution_bonus)

   3   2   3   1   3   2
1 |6| |4| |1| |2| |3| |5| 3
3 |4| |2| |5| |3| |6| |1| 3
3 |1| |3| |2| |6| |5| |4| 1
2 |2| |5| |3| |1| |4| |6| 2
2 |3| |6| |4| |5| |1| |2| 3
3 |5| |1| |6| |4| |2| |3| 2
   2   3   2   2   3   1
